In [ ]:
import numpy as np
from scipy.linalg import toeplitz

def construct_block_toeplitz(blocks, N):
    """
    Constructs an N x N block Toeplitz matrix where each block is a submatrix.

    Parameters:
    - blocks: List of m x m matrices defining the first row of the block Toeplitz.
    - N: Number of blocks in each dimension (matrix will be (N*m) x (N*m)).

    Returns:
    - Block Toeplitz matrix of size (N*m) x (N*m).
    """
    m = blocks[0].shape[0]  # Block size
    T = np.zeros((N * m, N * m))  # Initialize full matrix

    # Fill the Toeplitz structure using block matrices
    for i in range(N):
        for j in range(N):
            if i >= j:
                T[i*m:(i+1)*m, j*m:(j+1)*m] = blocks[i-j]
            else:
                T[i*m:(i+1)*m, j*m:(j+1)*m] = blocks[j-i].T  # Symmetric property

    return T

def apply_compact_correction(T, u, v):
    """
    Applies a compact low-rank correction to a Block Toeplitz matrix.

    Parameters:
    - T: The original block Toeplitz matrix.
    - u, v: Vectors (or blocks) to compute the rank-1 update ΔT = u v^T.

    Returns:
    - Updated matrix T'.
    """
    return T + np.outer(u, v)  # Efficient rank-1 update

# Example: Constructing a 3x3 Block Toeplitz matrix with 2x2 blocks
m = 2  # Block size
N = 3  # Number of blocks

# Define the first row of block matrices
blocks = [
    np.array([[2, 1], [1, 3]]),  # Main diagonal block
    np.array([[0, 1], [1, 0]]),  # First off-diagonal
    np.array([[1, 0], [0, 1]])   # Second off-diagonal
]

# Construct block Toeplitz matrix
T = construct_block_toeplitz(blocks, N)

# Define a low-rank correction u and v
u = np.random.randn(N*m)
v = np.random.randn(N*m)

# Apply compact relaxation
T_updated = apply_compact_correction(T, u, v)

# Print results
print("Original Block Toeplitz Matrix:\n", T)
print("\nUpdated Matrix After Compact Correction:\n", T_updated)

In [ ]:
import numpy as np
from scipy.sparse import block_diag

def construct_sparse_block_toeplitz(blocks, N):
    """
    Constructs an efficient block Toeplitz matrix representation 
    without explicitly storing the full matrix.
    
    Parameters:
    - blocks: List of m x m matrices defining the first row.
    - N: Number of blocks in each dimension.

    Returns:
    - A function that performs fast Toeplitz matrix-vector multiplication.
    """
    m = blocks[0].shape[0]  # Block size
    row_blocks = [blocks[i] for i in range(N)]
    col_blocks = [blocks[i].T for i in range(N)]

    def toeplitz_mv(x):
        """ Fast matrix-vector multiplication for structured Toeplitz """
        y = np.zeros(N * m)
        for i in range(N):
            y[i*m:(i+1)*m] = sum(blocks[j] @ x[(i-j)*m:(i-j+1)*m] for j in range(N) if i >= j)
        return y
    
    return toeplitz_mv

# Example Usage
m = 2
N = 3
blocks = [np.random.randn(m, m) for _ in range(N)]

fast_toeplitz_mv = construct_sparse_block_toeplitz(blocks, N)
x = np.random.randn(N * m)  # Input vector
y = fast_toeplitz_mv(x)  # Fast multiplication

print("Fast Toeplitz Multiplication Result:\n", y)

In [ ]:
from scipy.linalg import circulant, toeplitz
from scipy.fft import fft, ifft

def fast_toeplitz_mv_fft(blocks, x):
    """
    Uses FFT to compute Toeplitz matrix-vector multiplication efficiently.

    Parameters:
    - blocks: First row of block Toeplitz matrix.
    - x: Input vector.

    Returns:
    - Result of T @ x using FFT.
    """
    N = len(blocks)
    m = blocks[0].shape[0]

    # Construct circulant embedding (zero-pad and mirror)
    c = np.concatenate([blocks, np.zeros_like(blocks)], axis=0)
    fft_c = fft(c, axis=0)  # FFT of circulant extension

    # Apply FFT-based matrix-vector multiplication
    fft_x = fft(np.pad(x, (0, N*m-len(x)), 'constant'))
    result = ifft(fft_c * fft_x).real[:N*m]  # Extract first N*m elements

    return result

# Example Usage
blocks = [np.random.randn(2, 2) for _ in range(3)]
x = np.random.randn(6)  # Input vector
y_fft = fast_toeplitz_mv_fft(blocks, x)

print("FFT-based Toeplitz Multiplication Result:\n", y_fft)

In [ ]:
from scipy.linalg import svd

def apply_low_rank_correction(T, u, v, rank=2):
    """
    Applies a low-rank compact correction using truncated SVD.

    Parameters:
    - T: Original matrix.
    - u, v: Low-rank correction vectors.
    - rank: Number of singular values to retain.

    Returns:
    - Updated matrix T' with low-rank correction.
    """
    correction = np.outer(u, v)  # Rank-1 update
    U, S, Vh = svd(correction, full_matrices=False)
    S[rank:] = 0  # Truncate singular values
    correction_approx = U @ np.diag(S) @ Vh  # Reconstruct low-rank matrix

    return T + correction_approx

# Example Usage
T = np.random.randn(6, 6)
u, v = np.random.randn(6), np.random.randn(6)
T_updated = apply_low_rank_correction(T, u, v)

print("Low-Rank Updated Matrix:\n", T_updated)

In [ ]:
import cupy as cp

# Convert NumPy arrays to CuPy
T_gpu = cp.array(T)
u_gpu = cp.array(u)
v_gpu = cp.array(v)

# Perform GPU-accelerated update
T_updated_gpu = T_gpu + cp.outer(u_gpu, v_gpu)